In [1]:
import os

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import multiprocess

from datasets import load_dataset

from pytest_runner import run_pytest
from agent import CodeAgent
from client_interface.client import ClientInterface

In [2]:
dataset = load_dataset("mbpp", split="all")
dataset

Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 974
})

In [3]:
# dataset.to_csv("benchmark_data.csv", index=False)
# pd.read_csv("benchmark_data.csv", index_col=0)

In [10]:
#TODO: generate the initial responses first, then the feedbacks

class HumanEvalTestBenchmark:
    def __init__(self, dataset=dataset, tmp_folder="tmp", max_retries=8, max_feedback=1):
        self.dataset = dataset["code"][:5]#load_dataset("mbpp", split="all")["code"]
        self.tmp_folder = tmp_folder
        self.max_retries = max_retries
        self.max_feedback = max_feedback
        # self.client = ClientInterface()
    
    def _run_test(self, source_code, num_retries=5, max_feedback=1): # 3, 3
        from agent import CodeAgent
        from client_interface.client import ClientInterface
        from pytest_runner import run_pytest
        import pandas as pd

        result_passes = lambda result:\
                (result["coverage"] >= 90 and\
                (not result["failed_assertions"]) and\
                (not result["stderr"]))
        
        has_errors = lambda result:\
                (result["failed_assertions"]) or\
                (not result["stderr"])

        all_results = []
        model = CodeAgent(client=ClientInterface())#self.client)

        initial_code = model.generate_response(source_code, n=num_retries)


        return source_code, initial_code


    def run_evaluation(self):
        runs = []

        with multiprocess.Pool(processes=15) as pool: # multiprocess.cpu_count()
            runs = list(tqdm(pool.imap(self._run_test, self.dataset), total=len(self.dataset)))


        # identical = [self.dataset[0] for _ in range(4)]
        # for i in tqdm(identical):
        #     runs.append(self._run_test(i))
        # runs = self._run_test(self.dataset[0])


        return runs



evaluator = HumanEvalTestBenchmark()
ret = evaluator.run_evaluation()

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
source_code, initial_code = ret[0]

print(source_code)
initial_code

class Pair(object): 
	def __init__(self, a, b): 
		self.a = a 
		self.b = b 
def max_chain_length(arr, n): 
	max = 0
	mcl = [1 for i in range(n)] 
	for i in range(1, n): 
		for j in range(0, i): 
			if (arr[i].a > arr[j].b and
				mcl[i] < mcl[j] + 1): 
				mcl[i] = mcl[j] + 1
	for i in range(n): 
		if (max < mcl[i]): 
			max = mcl[i] 
	return max


["import pytest\nimport os\nimport sys\nsys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))\nfrom source import Pair, max_chain_length\n\ndef test_max_chain_length():\n    arr = [Pair(1,2), Pair(2,3), Pair(3,4), Pair(1,5)]\n    assert max_chain_length(arr, len(arr)) == 3\n\ndef test_max_chain_length_empty_input():\n    arr = []\n    assert max_chain_length(arr, len(arr)) == 0\n\ndef test_max_chain_length_single_element():\n    arr = [Pair(1,2)]\n    assert max_chain_length(arr, len(arr)) == 1",
 'import source\n\ndef test_max_chain_length():\n    arr = [source.Pair(2, 3), source.Pair(5, 1), source.Pair(3, 4)]\n    n = 3\n    assert source.max_chain_length(arr, n) == 3',
 'import source\nimport pytest\n\ndef test_max_chain_length():\n    arr = [source.Pair(1, 5), source.Pair(2, 7), source.Pair(3, 10)]\n    n = len(arr)\n    assert source.max_chain_length(arr, n) == 2\n\ndef test_max_chain_length_empty():\n    arr = []\n    n = len(arr)\n    assert source.m

In [ ]:

result_passes = lambda result:\
        (result["coverage"] >= 90 and\
        (not result["failed_assertions"]) and\
        (not result["stderr"]))

has_errors = lambda result:\
        (result["failed_assertions"]) or\
        (not result["stderr"])

all_results = []

In [2]:
x = ClientInterface()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [10]:
#TODO: generate the initial responses first, then the feedbacks

class HumanEvalTestBenchmark:
    def __init__(self, dataset=dataset, tmp_folder="tmp", max_retries=3, max_feedback=3):
        self.dataset = dataset["code"][:16]#load_dataset("mbpp", split="all")["code"]
        self.tmp_folder = tmp_folder
        self.max_retries = max_retries
        self.max_feedback = max_feedback
    
    def _run_test(self, prompt, num_retries=1, max_feedback=1): # 3, 3
        from agent import CodeAgent
        from pytest_runner import run_pytest
        import pandas as pd
        source_code = prompt

        result_passes = lambda result:\
                (result["coverage"] >= 90 and\
                (not result["failed_assertions"]) and\
                (not result["stderr"]))
        
        has_errors = lambda result:\
                (result["failed_assertions"]) or\
                (not result["stderr"])

        all_results = []
        for run_id in range(num_retries):
            prompt = source_code
            cnt_run = {"initial_code": prompt}
            model = CodeAgent()
            for idx_num in range(max_feedback):
                pytest_code, full_response = model.generate_response(prompt).values()
                pytest_code = "from source import *\n" + pytest_code
                result = run_pytest(source_code, pytest_code, random_subdir=True, clean_test=True) #TODO: SET TO TRUE TRUE FOR FULL RUN
                all_results.append([run_id, idx_num, result["coverage"], len(result["fails"]), result["stdout"], result["stderr"], pytest_code, source_code])
                # cnt_run[i] = {
                #     "generated_code" : pytest_code,
                #     "pytest_result" : result,
                #     "prompt" : prompt
                # }

                if result_passes(result) and idx_num ==0:
                    all_results = pd.DataFrame(all_results)
                    all_results.columns = ["run_id", "idx_num", "coverage", "num_fails", "stdout", "stderr", "pytest_code", "source_code"]
                    return all_results

                if result_passes(result):
                    break
                
                if has_errors:
                    # prompt = f"ASSERTION ERRORS:\n{result['fails']}\nSTDERR:\n{result['stderr']}"
                    prompt = f"STDOUT:\n{result['stdout']}\nSTDERR:\n{result['stderr']}"
                else:
                    prompt = f"Coverage is {result['coverage']}, try to reach 100% coverage.\nREPORT:\n{result['stdout']}"
        
            cnt_run["model_hist"] = model.history
            # all_results.append(cnt_run)

        all_results = pd.DataFrame(all_results)
        all_results.columns = ["run_id", "idx_num", "coverage", "num_fails", "stdout", "stderr", "pytest_code", "source_code"]

        return all_results


    def run_evaluation(self, model: CodeAgent):
        # self.model = model
        runs = []

        # with multiprocess.Pool(processes=multiprocess.cpu_count()) as pool: # TODO: LOCK NUMBER
        #     runs = list(tqdm(pool.imap(self._run_test, self.dataset), total=len(self.dataset)))


        identical = [self.dataset[0] for _ in range(4)]
        for i in tqdm(identical):
            runs.append(self._run_test(i))
        # runs = self._run_test(self.dataset[0])


        return runs



evaluator = HumanEvalTestBenchmark()


In [11]:
res = evaluator.run_evaluation(model=CodeAgent)

  0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
print(res[0].source_code.values[0])

class Pair(object): 
	def __init__(self, a, b): 
		self.a = a 
		self.b = b 
def max_chain_length(arr, n): 
	max = 0
	mcl = [1 for i in range(n)] 
	for i in range(1, n): 
		for j in range(0, i): 
			if (arr[i].a > arr[j].b and
				mcl[i] < mcl[j] + 1): 
				mcl[i] = mcl[j] + 1
	for i in range(n): 
		if (max < mcl[i]): 
			max = mcl[i] 
	return max


In [20]:
print(res[3].source_code.values[0])

class Pair(object): 
	def __init__(self, a, b): 
		self.a = a 
		self.b = b 
def max_chain_length(arr, n): 
	max = 0
	mcl = [1 for i in range(n)] 
	for i in range(1, n): 
		for j in range(0, i): 
			if (arr[i].a > arr[j].b and
				mcl[i] < mcl[j] + 1): 
				mcl[i] = mcl[j] + 1
	for i in range(n): 
		if (max < mcl[i]): 
			max = mcl[i] 
	return max


In [9]:
df = res[0]
df

,run_id,idx_num,coverage,num_fails,stdout,stderr,pytest_code,source_code
0,0,0,20.0,0,============================= test session sta...,,from source import *\ndef max_chain_length(arr...,"class Pair(object): \r\n\tdef __init__(self, a..."
1,0,1,100.0,73,============================= test session sta...,,from source import *\nimport pytest\r\nfrom so...,"class Pair(object): \r\n\tdef __init__(self, a..."
2,0,2,20.0,0,============================= test session sta...,,from source import *\ndef max_chain_length(arr...,"class Pair(object): \r\n\tdef __init__(self, a..."
3,1,0,20.0,0,============================= test session sta...,,from source import *\nclass Pair(object): \n ...,"class Pair(object): \r\n\tdef __init__(self, a..."
4,1,1,100.0,0,============================= test session sta...,,from source import *\ndef test_max_chain_lengt...,"class Pair(object): \r\n\tdef __init__(self, a..."
5,2,0,100.0,0,============================= test session sta...,,from source import *\nimport pytest\nfrom sour...,"class Pair(object): \r\n\tdef __init__(self, a..."


In [8]:
df = df[df.run_id == 0]
df

,run_id,idx_num,coverage,num_fails,stdout,stderr,pytest_code,source_code
0,0,0,100.0,138,============================= test session sta...,,from source import *\nimport pytest\nfrom sour...,"class Pair(object): \r\n\tdef __init__(self, a..."
1,0,1,100.0,138,============================= test session sta...,,from source import *\nimport pytest\nfrom sour...,"class Pair(object): \r\n\tdef __init__(self, a..."
2,0,2,100.0,138,============================= test session sta...,,from source import *\nimport pytest\nfrom sour...,"class Pair(object): \r\n\tdef __init__(self, a..."


In [9]:
df.pytest_code == df.pytest_code[0]

0    True
1    True
2    True
Name: pytest_code, dtype: bool

In [3]:
from huggingface_hub import InferenceClient


In [4]:

client = InferenceClient(model="http://127.0.0.1:8080")
client.text_generation(prompt="Write a code for snake game", max_new_tokens=2000)

"\nTo create a snake game, you will need to use a loop to move the snake's head and body. You can use a variable to keep track of the snake's position and update it as the snake moves. You can also use a variable to keep track of the snake's length and update it as the snake moves. Here is an example code snippet:\n\n```python\nimport turtle\n\ndef move_snake(x, y, length):\n    if x < 0:\n        x = -x\n    if y < 0:\n        y = -y\n    if x > 0:\n        x = 0\n    if y > 0:\n        y = 0\n    if x == 0:\n        x = length\n    if y == 0:\n        y = length\n    turtle.forward(x)\n    turtle.left(y)\n    turtle.right(x)\n    turtle.backward(y)\n    turtle.right(x)\n    turtle.left(y)\n    turtle.forward(x)\n    turtle.left(y)\n    turtle.right(x)\n    turtle.backward(y)\n    turtle.left(x)\n    turtle.forward(x)\n    turtle.left(y)\n    turtle.right(x)\n    turtle.backward(y)\n    turtle.left(x)\n    turtle.forward(y)\n    turtle.left(x)\n    turtle.right(y)\n    turtle.backward

In [5]:
import concurrent.futures
from tqdm.notebook import tqdm

batch_text = ["Write a code for snake game", "Write a code for an angry birds game", "Hello how are you", "What is the meaning of life?"] * 4

def gen_text(text, client=client): 
    return client.text_generation(prompt=text, max_new_tokens=2000)


# Using ThreadPoolExecutor with progress feedback
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    # Prepare futures and progress bar setup
    futures = [executor.submit(gen_text, text) for text in batch_text]
    out = []
    
    # Process as each future completes
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(batch_text)):
        result = future.result()
        out.append(result)

  0%|          | 0/16 [00:00<?, ?it/s]

: 

In [ ]:
#01:33

In [9]:
out

[" doing today?\nI'm doing well, thank you. How about you?\nI'm doing well, thank you. How about you?",
 " doing today?\nI'm doing well, thank you. How about you?\nI'm doing well, thank you. How about you?",
 '\nThe meaning of life is a philosophical question that has been debated for centuries. Some believe it is to find happiness, others believe it is to fulfill a purpose, while others believe it is to make a positive impact on the world. Ultimately, the meaning of life is subjective and can vary from person to person.',
 '\nThe meaning of life is a philosophical question that has been debated for centuries. Some believe it is to find happiness, others believe it is to fulfill a purpose, while others believe it is to make a positive impact on the world. Ultimately, the meaning of life is subjective and can vary from person to person.',
 " doing today?\nI'm doing well, thank you. How about you?\nI'm doing well, thank you. How about you?",
 '\nThe meaning of life is a philosophical que

In [3]:
import requests
import json

In [1]:
url = "http://localhost:8000/"

# The data you want to send in your POST request.
data = {
    "key1": "value1",
    "key2": "value2"
}

# Convert your data to JSON format.
data_json = json.dumps(data)

# Make the POST request, including the json data and appropriate headers.
response = requests.post(url, data=data_json, headers={"Content-Type": "application/json"})

# Check the response (optional).
print(response.text)

NameError: name 'json' is not defined

In [30]:
from openai import OpenAI
from constants import EVALUATION_SYS_PROMPT, dummy_code


# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "not_needed"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

#"TheBloke/WizardCoder-Python-13B-V1.0-AWQ" "TheBloke/WizardCoder-Python-34B-V1.0-AWQ", "m-a-p/OpenCodeInterpreter-CL-7B"
# "cognitivecomputations/dolphincoder-starcoder2-7b"  "TheBloke/WizardCoder-Python-34B-V1.0-AWQ"
response = client.chat.completions.create(model= "m-a-p/OpenCodeInterpreter-CL-7B",
                                    #   prompt=["San Francisco is a"],
                                       
  messages=[
    {"role": "system", "content": EVALUATION_SYS_PROMPT},
    {"role": "user", "content": dummy_code}],
    n=8,
    stream=False)
# print("Completion result:", completion)

In [50]:
import json
import requests

json.loads(requests.get("http://localhost:1234/v1/models", timeout=0.2).text)#["data"][0]["id"]

{'data': [{'id': 'Loaded from Chat UI',
   'object': 'model',
   'owned_by': 'organization-owner',
   'permission': [{}]}],
 'object': 'list'}

In [32]:
print(response.choices[7].message.content)

 Here's a way to create a test file for the given code using pytest:

```python:test.py
import sys
sys.path.append(".")  # add the source.py file in the same directory
from source import maxCntRemovedfromArray

def test_maxCntRemovedfromArray():
    arr = [1, 2, 4, 6]
    brr = [7]
    N = len(arr)
    M = len(brr)
    assert maxCntRemovedfromArray(arr, N, brr, M) == 1
```

To run the test with pytest:

```shell
pytest -v
```

This will automatically discover the test and run it, giving you output like:

```shell
...
PASSED test.py::test_maxCntRemovedfromArray - maxCntRemovedfromArray(arr, N, brr, M)
```

If the function implementation in `source.py` is correct, you should see the above test pass. 

If there were any errors in `source.py`, pytest would give you line numbers and details about what failed. You would then need to update `test.py` to properly test the revised `source.py`.

If you want to increase code coverage, you might consider adding more tests, including different inpu

In [22]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

messages=[
{"role": "system", "content": EVALUATION_SYS_PROMPT},
{"role": "user", "content": dummy_code}]

completion = client.chat.completions.create(
    model="local-model", # this field is currently unused
    messages=messages,
    temperature=0.7,
    stream=False,
)

new_message = {"role": "assistant", "content": ""}
    
    
print(completion.choices[0].message.content)


Here's the test file for the given code using pytest framework. 

```python
import pytest
from source import maxCntRemovedfromArray

def test_maxCntRemovedfromArray():
    arr = [1, 2, 4, 6]
    brr = [7]
    N = len(arr)
    M = len(brr)
    assert maxCntRemovedfromArray(arr, N, brr, M) == 3
```

To run the test file, save it as `test_source.py` and execute the following command in your terminal:


```bash
pytest test_source.py
```

This should give you a passing test case with 100% coverage. To increase code coverage, we can add more test cases to the file. For example:


```python
import pytest
from source import maxCntRemovedfromArray

def test_maxCntRemovedfromArray():
    arr = [1, 2, 4, 6]
    brr = [7]
    N = len(arr)
    M = len(brr)
    assert maxCntRemovedfromArray(arr, N, brr, M) == 3

def test_maxCntRemovedfromArray_same_arrays():
    arr = [1, 2, 3]
    brr = [1, 2, 3]
    N = len(arr)
    M = len(brr)
    assert maxCntRemovedfromArray(arr, N, brr, M) == 0

def test_maxC

In [23]:
import asyncio
from openai import OpenAI

# Assuming `openai.OpenAI` has an asynchronous equivalent or a way to be used asynchronously.
# If not, you'll need to adapt this to run the synchronous code in an asynchronous event loop.

# Define an asynchronous function to make the API call
async def async_api_call(client, messages):
    # Simulate an asynchronous call to the API
    # In practice, you would use an asynchronous method of the client here
    completion = await asyncio.get_event_loop().run_in_executor(None, client.chat.completions.create,
        model="local-model", # this field is currently unused
        messages=messages,
        temperature=0.7,
        stream=False,
    )
    return completion.choices[0].message.content

# Create an instance of the OpenAI client
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
messages = [
    {"role": "system", "content": "EVALUATION_SYS_PROMPT"},
    {"role": "user", "content": "dummy_code"}
]

# Define the main coroutine that runs both API calls asynchronously
async def main():
    # Run two API calls concurrently and wait for both to finish
    results = await asyncio.gather(
        async_api_call(client, messages),
        async_api_call(client, messages),
    )
    
    # Print the results
    for result in results:
        print(result)

# Run the main coroutine
asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [6]:
import re
text = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:

        print(chunk.choices[0].delta.content, end="")

        if re.findall(r"```(?:\n)?(?:python)?(.*?)```", text, re.DOTALL) or\
            "__name__ ==" in text:
            break
        # text = text + chunk.choices[0].delta.content


# print(text)






""<"S|
orryim
 for_import thesep py confusion|test,>
 I import need sys you
 to
 Mreatedef a max pyCtestnt fileRem foroved thisfrom codeArray.( Herearr',s N the, updated br coder:,
):

sort`` arrpython.
)importreverse py=testFalse

from    source i import = max C0nt
 em   oved sumfromArrArray =
0

   def for test i_ inmax rangeC(ntNRem):oved
 +=m       Array sum():Arr
 arr   [ arri =] [
1   , sum Br2r, =  40,
 in 6 for] i
 range   ( brMr): =
r[       7 sum]Br
 +=    br Nr =[ leni(]arr
   
 c   nt MRem =E lenlem( =br r0)

       while assert i max <C Nnt andRem sumovedArrfrom >=Array sum(Brarrr,: N
,        br sumrArr, -= M arr)[ ==i ]3
       
 i``` +=
1

       You if can sum runArr this >= code sum usingBr pyrtest: to
 see            if c itnt passesRem.E Iflem there +=' s1 any
 error   , return please c provident theRem traceEbacklem and
 I
'defll test help_ youmax fixC itnt.Rem ovedfromArray():
    arr = [1, 2, 4, 6]
    brr = [7]
    N = len(arr)
    M = len(brr)
    assert m

KeyboardInterrupt: 

In [36]:
print(response.choices[1].message.content)

"""

def add_numbers(a: int, b: int) -> int:
    return a + b

# Test the function with some examples
print(add_numbers(2, 3)) # Output: 5
print(add_numbers(5, 7)) # Output: 12
print(add_numbers(-2, 8)) # Output: 6
print(add_numbers(0, 0)) # Output: 0 
